### 1. Problems Encountered in the Map

For this project I had initially chosen data from Brasilia, which is the political capital of Brazil and also my hometown. But an initial analysis showed very little information available for Brasilia (there were zero entries with address data). 

I've then switched to Rio de Janeiro, the second largest city in Brazil, known for its beautiful beaches and carnival celebrations. According to wikipedia, the municipality of Rio de Janeiro has an area of 1.2 million square kilometers, and 6.5 million inhabitants(12 million when you include metropolitan area)

The data was download from Mapzen, size file is 243 mb. The code in wrangling.py is being used to clean up the data, and convert to a json format ready, document database ready (code is based on lesson 6). 

Let's import the data, and take a first look at it.

In [156]:
# -*- coding: utf-8 -*-
from __future__ import print_function
from pprint import pprint
import random
import re 
from collections import *

%run wrangling.py

#setup the connection
from pymongo import MongoClient
client = MongoClient()
db = client['osm']
coll = db['rio-de-janeiro']

In [157]:
# process data, save to json and import data using shell (faster)
process_map('rio-de-janeiro_brazil.osm')
db['rio-de-janeiro'].drop()
!mongoimport --db osm --collection rio-de-janeiro --file rio-de-janeiro_brazil.osm.json

2015-10-26T23:52:10.827-0200	connected to: localhost
2015-10-26T23:52:13.821-0200	[###.....................] osm.rio-de-janeiro	42.5 MB/276.3 MB (15.4%)
2015-10-26T23:52:16.821-0200	[#######.................] osm.rio-de-janeiro	87.1 MB/276.3 MB (31.5%)
2015-10-26T23:52:19.822-0200	[###########.............] osm.rio-de-janeiro	131.1 MB/276.3 MB (47.4%)
2015-10-26T23:52:22.824-0200	[###############.........] osm.rio-de-janeiro	174.8 MB/276.3 MB (63.3%)
2015-10-26T23:52:25.821-0200	[##################......] osm.rio-de-janeiro	218.4 MB/276.3 MB (79.1%)
2015-10-26T23:52:28.821-0200	[#######################.] osm.rio-de-janeiro	265.4 MB/276.3 MB (96.1%)
2015-10-26T23:52:29.484-0200	imported 1347275 documents


In [159]:
#let's run some queries to see how complete is the data
#check how many entries there are, and how many of those have address
cursor = coll.find()
print('Total entries: ', cursor.count())
cursor = coll.find({'address': {'$exists': 1}})
print('Total entries with address: ', cursor.count())
print(14921/1347275.0)

Total entries:  1347275
Total entries with address:  14921
0.0110749475794


There are not many entries with address, only 14,921 out of 1,347,275. That is a little over 1%. But is still far better than Brasilia, which had 0 entries with address. Let's take a look at the addresses, see if there is any fixing required:

In [160]:
#quick look at addresses to see overall format 
cursor = coll.find({'address.street': {'$exists': 1}}) 
for row in cursor.limit(3):
    for k,v in row['address'].items():
        print(k,": ",v)
    print("")
 

city :  Rio de Janeiro
country :  BR
state :  RJ
street :  Praça Senador Salgado Filho
postcode :  20021-340
continent :  South America

country :  BR
street :  Rua do Catete
housenumber :  153
postcode :  22220-000
city :  Rio de Janeiro

city :  Rio de Janeiro
street :  Rua do Catete
housenumber :  104
country :  BR



The information relevant to us is street number, house number and post code. Let's take a further look at these three. In portuguese, the first word of the street indicates its type (Rua means "Street", for example). So let's isolate all possible types of street and see if they are consistent.

In [161]:
cursor = coll.find({'address.street': {'$exists': 1}}) 
prefixes = Counter()
pattern = re.compile('[\w]*')
for row in cursor:
    prefix = re.search(pattern,row['address']['street']).group(0)
    prefixes[prefix] += 1 

#remove the ocurrences that appear only once
prefixes = {k:v for k,v in prefixes.items() if v>1}
print(prefixes)

{u'Boulevard': 5, u'Caminho': 28, u'Parque': 2, u'Ladeira': 13, u'Alameda': 5, u'Dias': 2, u'Estrada': 1549, u'Praca': 2, u'Rua': 8408, u'Praia': 80, u'Beco': 6, u'Campo': 2, u'Professor': 2, u'rua': 2, u'M': 5, u'P': 2, u'R': 7, u'Alfredo': 5, u'Largo': 20, u'Via': 20, u'Pra': 209, u'Rod': 8, u'Rodovia': 24, u'Travessa': 55, u'Av': 64, u'Avenida': 2568}


The data looks good, overall. But we could reduce the number of the known inconsistencies, such as lower case and abbreviatons. The following code traverses through the findings, and whenever there are known inconsistencies, replace them.




In [162]:
st_name_dict = {'rua': 'Rua', 'P': 'Praia', 'Pra': 'Praia', 'Rod': 'Rodovia', 'Av': 'Avenida', 'R': 'Rua'}
cursor = coll.find({'address.street': {'$exists': 1}}) 

#traverse through results. If found in dict, update
for row in cursor:
    street = row['address']['street']
    prefix = re.search(pattern,street).group(0)    
    if prefix in st_name_dict.keys():
        updated_st_name = street.replace(prefix, st_name_dict[prefix])
        coll.update({'_id': row['_id']}, {'$set': {'address.street': updated_st_name}})

In [163]:
#test
cursor = coll.find({'address.street': {'$exists': 1}}) 
prefixes = Counter()
pattern = re.compile('[\w]*')
for row in cursor:
    prefix = re.search(pattern,row['address']['street']).group(0)
    prefixes[prefix] += 1 

#remove the ocurrences that appear only once
prefixes = {k:v for k,v in prefixes.items() if v>1}
print(prefixes)

{u'Boulevard': 5, u'Estrada': 1549, u'Praca': 2, u'Campo': 2, u'Caminho': 28, u'Parque': 2, u'Rua': 8417, u'M': 5, u'Rodovia': 32, u'Praia': 291, u'Ladeira': 13, u'Via': 20, u'Alfredo': 5, u'Beco': 6, u'Travessa': 55, u'Professor': 2, u'Largo': 20, u'Avenida': 2632, u'Alameda': 5, u'Dias': 2}


Now let's look at post code. The correct pattern in Brazil is xxxxx-xxx. Let's first look for the inconsistencies.

In [164]:
#check for inconsistencies on postcode
cursor = coll.find({'address.postcode': {'$exists': 1}}) 
pattern = re.compile('\d{5}-\d{3}')
inconsistencies = []
for row in cursor:
    postcode = row['address']['postcode']
    if not re.search(pattern,postcode):
        inconsistencies.append(postcode)

print(inconsistencies)

[u'22240004', u'20270060', u'25660004', u'22280030', u'22270010', u'24346030', u'2261001', u'24324270', u'22410000', u'22281020', u'20510180', u'25010060', u'26130130', u'26130130', u'26130130', u'26130130', u'26130130', u'25.620-003', u'20720010', u'25963082', u'26953201', u'25958060', u'25953671', u'25955240', u'35953060', u'2695307', u'24230', u'23025 520', u'20770240', u'20775090', u'20550200', u'20550200', u'24744520', u'21921000', u'22440040', u'22440040', u'22.776-070', u'26130230', u'26130230', u'21311050', u'20745000', u'26900000', u'24020000', u'21941005', u'21515090', u'20090-00', u'22430090', u'24220480', u'24060037', u'24060037', u'24060037', u'24060037', u'21230354', u'22631030', u'20770\u2011001', u'22440033', u'22750009', u'20216005', u'52645100', u'2246-000', u'22471340', u'22471340', u'22471340', u'22471270', u'22471340', u'22471220', u'22471220', u'22471220', u'22471340', u'22471340', u'22471340', u'22471350', u'21920225', u'22071100', u'22795255', u'22735030', u'227

There are many variations. Mostly are just numbers, without the dash between the fifth and sixth digit. Some are missing a number, and some use a different notation with a dot between the second and third number.

To fix this, we can strip everything which is not a number, and then join together with a dash at the appropriate position. If there is less or more than 8 digits, we shall delete the attribute.

In [165]:
#check for inconsistencies on postcode
cursor = coll.find({'address.postcode': {'$exists': 1}}) 
for row in cursor:
    original = row['address']['postcode']
    digits = re.findall('\d', original)
    length = len(digits)
    digits.insert(5, '-')
    new = ''.join(digits)
    if length != 8:
        updated = coll.update({'_id': row['_id']}, {'$unset': {'address.postcode': ''}})        
    elif new != original:
        updated = coll.update({'_id': row['_id']}, {'$set': {'address.postcode': new}})

In [166]:
#test
cursor = coll.find({'address.postcode': {'$exists': 1}}) 
pattern = re.compile('\d{5}-\d{3}')
inconsistencies = []
for row in cursor:
    postcode = row['address']['postcode']
    if not re.search(pattern,postcode):
        inconsistencies.append(postcode)

print(inconsistencies)

[]


House numbers are more complicated to handle. It may contain numbers, letters and symbols in different variations, and so there is not a pattern we can aim for. With that in mind, we will leave house numbers unchanged.

As a final check, let's take a look at the main streets and post codes, and see if they look consistent.

In [169]:
pipeline = [
    {'$match': {'address.postcode': {'$exists': 1}} },
    {'$group': {'_id': '$address.postcode', 'count': {'$sum': 1} } },
    {'$sort': {'count': -1} },
    {'$limit': 5}
]
groupby_postalcode = [doc for doc in coll.aggregate(pipeline)]

pipeline = [
    {'$match': {'address.street': {'$exists': 1}} },
    {'$group': {'_id': '$address.street', 'count': {'$sum': 1} } },
    {'$sort': {'count': -1} },
    {'$limit': 5}
]
groupby_street = [doc for doc in coll.aggregate(pipeline)]

print('Main post codes')
print(groupby_postalcode)
print('Main streets')
print(groupby_street)

Main post codes
[{u'count': 96, u'_id': u'22720-410'}, {u'count': 84, u'_id': u'22471-003'}, {u'count': 76, u'_id': u'22720-400'}, {u'count': 62, u'_id': u'22220-000'}, {u'count': 56, u'_id': u'22221-000'}]
Main streets
[{u'count': 511, u'_id': u'Estrada dos Bandeirantes'}, {u'count': 498, u'_id': u'Avenida das Am\xe9ricas'}, {u'count': 175, u'_id': u'Avenida dos Mananciais'}, {u'count': 153, u'_id': u'Avenida L\xfacio Costa'}, {u'count': 122, u'_id': u'Avenida Genaro de Carvalho'}]


The data on streets and postcodes do not have outliers. There are a number of postal codes and streets with only one address, but it is highly possible that represents the lack of available information, rather than an input error. 

Finally, let's look at which tags there are available. Let's go through each element of the collection, and count the ocurrences of each type of tag. This information will guide the next step, data analysis, by showing us exactly what information we can extract from this dataset.

We will divide in three groups: general attributes, address attributes and created attributes

In [170]:
cursor = coll.find() 
address_attrib = Counter()
created_attrib = Counter()
general_attrib = Counter()

for row in cursor:
    for elem in row.keys():
        if elem == 'address':
            for subelem in row[elem]:
                address_attrib[subelem] += 1
        elif elem == 'created':
            for subelem in row[elem]:
                created_attrib[subelem] += 1            
        else:
            general_attrib[elem] += 1


In [148]:
#filter and print only those greater than 1000
treshold = 1000
print('General')
pprint({k:v for k,v in general_attrib.items() if v>treshold})
print('Created')
pprint({k:v for k,v in created_attrib.items() if v>treshold})
print('Address:')
pprint({k:v for k,v in address_attrib.items() if v>treshold})


General
{u'IPP:BAIRRO': 1253,
 u'IPP:CHAVE': 1221,
 u'IPP:CODATIVIDA': 1256,
 u'IPP:CODBAIRRO': 1252,
 u'IPP:CODLINFERR': 1188,
 u'IPP:COD_INEP': 1251,
 u'IPP:COD_SMA': 1222,
 u'IPP:CRE': 1256,
 u'IPP:CodFavela': 1013,
 u'IPP:DESIGNACAO': 1256,
 u'IPP:ENDNOVO': 1256,
 u'IPP:FLG_VALIDA': 1137,
 u'_id': 1347275,
 u'id': 1347275,
 u'node_refs': 132366,
 u'pos': 1214909,
 u'type': 1347275}
Created
{u'changeset': 1347275,
 u'timestamp': 1347275,
 u'uid': 1347275,
 u'user': 1347275,
 u'version': 1347275}
Address:
{u'city': 3238,
 u'country': 1422,
 u'housenumber': 11111,
 u'inclusion': 1399,
 u'interpolation': 1683,
 u'postcode': 2344,
 u'street': 13119}


### Data Overview                                                
This section contains basic statistics about the dataset and the MongoDB queries used to gather them.
            

File sizes:                                                  
rio-de-janeiro_brazil.osm ......... 248 MB  
rio-de-janeiro_brazil.osm.json .... 276 MB  

In [171]:
# Number of documents                                                
coll.find().count()                                               

1347275

In [172]:
# Number of nodes                                                
coll.find({"type":"node"}).count()

1214909

In [173]:
# Number of ways
coll.find({"type":"way"}).count()                                   

132366

The division between nodes and ways is 90% to 10%.

In [174]:
#Number of posts with position
coll.find({"pos": {"$exists": 1}}).count()

1214909

In [175]:
#Number of posts with address
coll.find({"address": {"$exists": 1}}).count()

14921

In [176]:
# Number of unique users                                                
len(coll.distinct('created.user'))


977

In [177]:
# Top 10 contributing users
pipeline = [
        {"$group": {"_id":"$created.user", "count":{"$sum":1}}}, 
        {"$sort": {"count": -1}}, 
        {"$limit": 10}
        ]
cursor = coll.aggregate(pipeline)
posts_count = 0
for row in cursor:
    print(row)
    posts_count+=row['count']
    
print('total posts for top 10 users: ', posts_count)
    

{u'count': 379891, u'_id': u'Alexandrecw'}
{u'count': 178198, u'_id': u'AlNo'}
{u'count': 107435, u'_id': u'ThiagoPv'}
{u'count': 89013, u'_id': u'Import Rio'}
{u'count': 74735, u'_id': u'Geaquinto'}
{u'count': 69132, u'_id': u'Nighto'}
{u'count': 55617, u'_id': u'Thundercel'}
{u'count': 23687, u'_id': u'bmog'}
{u'count': 23248, u'_id': u'user_401472'}
{u'count': 23062, u'_id': u'Skippern'}
total posts for top 10 users:  1024018


In [178]:
# Number of users per number of posts
pipeline = [{"$group":{"_id":"$created.user", "count":{"$sum":1}}}, 
            {"$group":{"_id":"$count", "num_users":{"$sum":1}}}, 
            {"$sort":{"_id":1}}, 
            {"$limit":10}
           ]
cursor = coll.aggregate(pipeline)
users_count = 0
for row in cursor:
    print(row)
    users_count += row['num_users']

print('total users with 10 or less posts: ', users_count)

#_id represents post count


{u'num_users': 182, u'_id': 1}
{u'num_users': 86, u'_id': 2}
{u'num_users': 47, u'_id': 3}
{u'num_users': 32, u'_id': 4}
{u'num_users': 37, u'_id': 5}
{u'num_users': 33, u'_id': 6}
{u'num_users': 21, u'_id': 7}
{u'num_users': 22, u'_id': 8}
{u'num_users': 22, u'_id': 9}
{u'num_users': 15, u'_id': 10}
total users with 10 or less posts:  497


More than 50% of the users have 10 or less posts, and close to 20% have only one post. 

The top 10 contributing users list shows there is a high concentration of posts between few top users. The top 1st contributing user has 16 times more posts than the 8th contributing user. 

The top user alone is responsible for 28% of the total edits. The top10 are responsible for 76% of the total posts. Those numbers probably result from automated ways of inputting data into OSM database.

### Additional Ideas

There is few useful data available apart from latitude and longitude. 
Only 14,921, or 1,1% of the posts, have information on address. Other non basic tags are present in around a thousand posts each, which is less than 0,1% of the posts count. 

Open Street Map, although a great initiative, is not known in Brazil. This is the first time I've heard of it. 

Upon a first look at the website, I could not find an easy way to contribute. It would be of great help if the community had an easier way of mapping the data, possibly by a mobile app that captures as much information as possible (it can get geolocation from the phone gps, for example), and the user only has to fill in the missing parts.

As suggested in the example, this mobile app can be gamified to encourage usage, adding features such as points, ranking, badges, reputation, self promotion, and extra perks.

The benefits of improving OSM data for the developer, business owner and final user, should also be clearly stated. Any attempt of gathering more users should be preceded by a marketing campaign, starting on its own website, and usings its social net to spread.

### Additional data exploration using MongoDB queries

On the first part, we took a comprehensive look at the information available by mapping the number of ocurrences of each tag. In the result we can see that there are not a lot of tags that could be used to further explore the data and make inferences. 

Nevertheless, let's look at the most populated streets located on suburb, and average posts per street and postal code

In [179]:
# Most populated streets on suburban areas 
# based on address.suburb
pipeline = [
    {'$match': {'address.suburb': {'$exists': 1}, 'address.street': {'$exists': 1}}},
    {'$group': {'_id': '$address.street', 'count': {'$sum': 1}}},
    {'$sort': {'count': -1}},
    {'$limit': 5}
]

cursor = coll.aggregate(pipeline)
for row in cursor:
    print(row['_id'], ': ', row['count'])

Avenida das Américas :  57
Avenida Vieira Souto :  56
Rua do Senado :  20
Rua Vinícius de Moraes :  19
Avenida Embaixador Abelardo Bueno :  14


In [180]:
# Average posts per postal code
pipeline = [
    {'$match': {'address.postcode': {'$exists': 1}}},
    {'$group': {'_id': '$address.postcode', 'count': {'$sum': 1}}},
    {'$group': {'_id': 'average_posts_per_postcode', 'avg': {'$avg': '$count'}}},
]

cursor = coll.aggregate(pipeline)
for row in cursor:
    print(row['_id'], ': ', row['avg'])

average_posts_per_postcode :  3.20656634747


In [182]:
# Average posts per street
pipeline = [
    {'$match': {'address.street': {'$exists': 1}}},
    {'$group': {'_id': '$address.street', 'count': {'$sum': 1}}},
    {'$group': {'_id': 'average_posts_per_street', 'avg': {'$avg': '$count'}}},
]

cursor = coll.aggregate(pipeline)
for row in cursor:
    print(row['_id'], ': ', row['avg'])
    

average_posts_per_street :  6.75888717156


### Conclusion                        

As stated before, the available data is centered on geocoordinates. Any other data is residual, present in only around 1% of the posts.

The data was relatively clean, but thorough this project we made sure to further improve the address data by standardizing street names and postal codes.

A suggestion is given of promoting a mobile app that uses cell phone resources to facilitate data entry. That would encorage new contributors, even more if the Open Street Map is perceived as relevant by its audience. A well executed marketing campaign could booster OSM's awareness amongst users.